In [7]:
import os
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from dotenv import load_dotenv
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage
load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")

llm = ChatOpenAI(
    model="google/gemini-2.5-flash",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

tavily_client = TavilyClient()

In [4]:
@tool
def web_search(query: str) -> Dict[str, Any]:
    """
    Search the web for information
    """
    return tavily_client.search(query)

In [5]:
system_prompt = """ 
Вы — личный шеф-повар. Пользователь предоставит вам список ингредиентов, которые остались у него дома.

С помощью инструмента веб-поиска найдите в сети рецепты, для которых подойдут эти ингредиенты.

Если пользователь попросит, верните ему предложения по рецептам и, в конечном счёте, инструкции по их приготовлению.
"""

In [8]:
agent = create_agent(
    model=llm,
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="У меня осталась курица и рис. Что я могу приготовить?")]},
    config
)

print(response['messages'][-1].content)

Из курицы и риса можно приготовить множество блюд! Вот несколько идей:

*   **Курица терияки с рисом и овощами**
*   **Простая паэлья с курицей и овощами**
*   **Рис с курицей и зелёным горошком на сковороде**
*   **Запеканка с курицей, рисом и сырным соусом**
*   **Буррито-боул с курицей, рисом и фасолью**
*   **Ризотто с курицей и грибами**
*   **Индонезийский жареный рис с курицей**
*   **Плов с курицей**
*   **Курица с рисом и овощами в одной посуде**

Какой из этих вариантов звучит наиболее аппетитно? Или, может быть, вы хотите уточнить, какие еще ингредиенты у вас есть, чтобы я мог предложить более конкретный рецепт?


In [9]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='У меня осталась курица и рис. Что я могу приготовить?', additional_kwargs={}, response_metadata={}, id='ddb935e3-4a5a-4a91-858e-606e7d5e729e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 109, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 6.52e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 3.27e-05, 'upstream_inference_completions_cost': 3.25e-05}}, 'model_provider': 'openai', 'model_name': 'google/gemini-2.5-flash', 'system_fingerprint': None, 'id': 'gen-1767112760-sRwNILZVr0rFha9wjBid', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b702